In [ ]:
!pip install midi2audio
!pip install fluidsynth
!pip install --upgrade music21==6.7.1

!pip install gTTS 
!pip install pygobject
!pip install numpy
!pip install matplotlib
!pip install librosa
!pip install scipy
!pip install IPython
!pip install playsound 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4513 sha256=fb8dd589bd20177c0383a364f7b994260160a2cb47a4f9e138f00aa7fed4e094
  Stored in directory: /root/.cache/pip/wheels/44/4b/81/dee2d535b2dd27af13c230ba49aa6f05a5dff0f57c68acef55
Successfully built fluidsynth
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.2 MB 1.1 MB/s 
  Created wheel for music21: filename=music21-6.7.1-py3-none-any.whl size=21941719 sha256=4aff7a8a7b2c499d9f9f7b66b90af72efad04a4b8b4b0f991df09554e96897b3
  Stored in directory: /root/.cache/pip/wheels/72/44/61/90e4e65262ca1b4d9f707527b540729ce3f64e00fc6b38d54c
Successfully built music21
  Attempting uninstall: music21
    Found existing installation:

In [ ]:
# enables music21 to render images of musical notes
print('installing lilypond...')
!apt-get install lilypond > /dev/null

# converts midi files to wav files into order to play them
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

print('done!')

installing lilypond...
Extracting templates from packages: 100%
installing fluidsynth...
done!


In [ ]:
import music21
from music21 import *
from IPython.display import Image, Audio
import IPython
import pandas as pd

In [ ]:
c = music21.converter.parse('Sonata_in_G_3rd_mvt.mxl')
# play(c)
# c.show('musicxml.pdf')
# c.show('midi')
for i in c.getTimeSignatures():
  print(i)
print(c)
fp = c.write('midi', 'Sonata_in_G_3rd_mvt' + '.mid')
# show(c)

print('------')
for i in c:
  print(i)
  '''for j in i:
    print(j)'''

print(c.metadata.all())

<music21.meter.TimeSignature 4/4>
<music21.stream.Score 0x7f1915ae2190>
------
<music21.text.TextBox 'Sonata for...'>
<music21.text.TextBox 'Beethoven'>
<music21.text.TextBox '2'>
<music21.text.TextBox '3'>
<music21.stream.Part Violin>
<music21.layout.ScoreLayout>
[('composer', 'Beethoven'), ('copyright', 'Copyright ©'), ('movementName', 'Sonata_in_G_3rd_mvt.mxl'), ('title', 'Sonata for Violin in G, 3rd mvt.')]


In [ ]:

'''allBach = corpus.search('bach')
x = allBach[0]
p = x.parse()'''
tempo = 120
time_sig = .5 # ratio to 4-times note
note_list = []
duration_list = []

# print(c.recurse().notes)

# If it is a chord
'''for i in c.flat.notes[347]:
  print("Note: %s%d %0.01f" % (i.pitch.name, i.pitch.octave, i.duration.quarterLength))'''
for n in c.flat.notesAndRests:
  try:
    # print("Note: %s%d %0.01f" % (n.pitch.name, n.pitch.octave, n.duration.quarterLength))
    temp_name = n.pitch.name
    if n.duration.quarterLength == float(0.0): # it is a Ornaments
      temp_name = str('Ornaments ') + str(n.pitch.name)# O for Ornaments

    note_list.append(str(temp_name) + str(n.pitch.octave))
    duration_list.append(n.duration.quarterLength*(60/tempo)/time_sig) # *(60/tempo)/time_sig
  except:
    try:
      for i in n:
        None
      # print('------------------It is a chord:------------------')
      temp_note = []
      temp_duration = []

      for i in n:
        # print("Note: %s%d %0.01f" % (i.pitch.name, i.pitch.octave, i.duration.quarterLength))
        temp_note.append(str(i.pitch.name) + str(i.pitch.octave))
        temp_duration.append(i.duration.quarterLength*(60/tempo)/time_sig)
      # print('------------------------End------------------------')

      y = ''
      for i in temp_note:
        y += str(i)
        y = y + ','

      note_list.append(str('chord, ')+ y + str('chord end'))
      duration_list.append(temp_duration)

    except:
      # print("Rest Note: %0.01f" % (n.duration.quarterLength))
      try:
        note_list.append('Rest')
        duration_list.append(n.duration.quarterLength*(60/tempo)/time_sig)
      except:
        raise 'It is not a note, chord or rest'

# show(c)
print(len(note_list))
print(len(duration_list))
all_list = zip(note_list, duration_list)

1160
1160


In [ ]:
pd.set_option('display.max_rows', 1500)
df = pd.DataFrame()
df['Note type'] = note_list
df.insert(1, 'duration (in s)', duration_list)
print(df)

                       Note type  duration (in s)
0                           Rest              1.0
1                           Rest              2.0
2                           Rest              2.0
3                           Rest              2.0
4                           Rest              1.0
5                             D5              0.5
6                             D5              0.5
7                             D5              0.5
8                             D5              0.5
9                  Ornaments C#5              0.0
10                  Ornaments D5              0.0
11                            E5              0.5
12                           F#5              0.5
13                            G5              0.5
14                            B5             0.25
15                          Rest             0.25
16                            D5              0.5
17                            D5              0.5
18                            D5              0.5


In [ ]:
import gtts
%matplotlib inline
import seaborn
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
from playsound import playsound
import time

In [ ]:
print(note_list)

plt.rcParams['figure.figsize'] = (13, 5)
a = ''
for i in note_list[1145:]:
  a += str(i)
  a = a + '... | '

print(a)

t1 = gtts.gTTS(a) 

# save the audio file  
t1.save("test.wav")   

['Rest', 'Rest', 'Rest', 'Rest', 'Rest', 'D5', 'D5', 'D5', 'D5', 'Ornaments C#5', 'Ornaments D5', 'E5', 'F#5', 'G5', 'B5', 'Rest', 'D5', 'D5', 'D5', 'D5', 'Ornaments C#5', 'Ornaments D5', 'E5', 'F#5', 'G5', 'C5', 'B4', 'A4', 'G4', 'F#4', 'G4', 'A4', 'B4', 'C5', 'B4', 'A4', 'G4', 'F#4', 'G4', 'A4', 'B4', 'C5', 'B4', 'A4', 'G4', 'F#4', 'G4', 'A4', 'B4', 'C5', 'B4', 'C5', 'B4', 'C5', 'D4', 'G4', 'B4', 'D5', 'C#5', 'D5', 'E5', 'D5', 'B4', 'D5', 'C5', 'A4', 'G4', 'B4', 'D4', 'B4', 'D4', 'G4', 'B4', 'D5', 'C#5', 'D5', 'E5', 'D5', 'B4', 'D5', 'C5', 'A4', 'G4', 'D4', 'G4', 'B4', 'D5', 'C#5', 'D5', 'E5', 'D5', 'B4', 'D5', 'C5', 'A4', 'G4', 'B4', 'D4', 'B4', 'D4', 'G4', 'B4', 'D5', 'C#5', 'D5', 'E5', 'D5', 'B4', 'D5', 'C5', 'A4', 'G4', 'G4', 'B4', 'A4', 'B4', 'D4', 'G4', 'D4', 'F#4', 'Ornaments A4', 'G4', 'F#4', 'G4', 'B4', 'A4', 'B4', 'D4', 'G4', 'D4', 'F#4', 'C5', 'D5', 'B4', 'C5', 'A4', 'B4', 'G4', 'A4', 'F#4', 'F#4', 'G4', 'D5', 'Rest', 'C5', 'D5', 'B4', 'C5', 'A4', 'B4', 'G4', 'A4', 'F#4', 

In [ ]:
x, sr = librosa.load('test.wav')
ipd.Audio('test.wav')

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
